# LSTM vs STC Comparison

This notebook loads Microsoft stock data, trains an LSTM model to predict whether the next day's closing price will increase, and compares it with a simple Schaff Trend Cycle (STC) baseline.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Load Data

In [ ]:
df = pd.read_csv('MSFT_1986_2025-06-30.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)
df.head()

## Calculate STC

In [ ]:
def schaff_trend_cycle(close, fast_period=23, slow_period=50, cycle_period=10):
    ema_fast = close.ewm(span=fast_period, adjust=False).mean()
    ema_slow = close.ewm(span=slow_period, adjust=False).mean()
    macd = ema_fast - ema_slow
    lowest_macd = macd.rolling(window=cycle_period).min()
    highest_macd = macd.rolling(window=cycle_period).max()
    stoch_macd = 100 * (macd - lowest_macd) / (highest_macd - lowest_macd)
    stoch_macd_smoothed1 = stoch_macd.ewm(span=3, adjust=False).mean()
    stoch_macd_smoothed2 = stoch_macd_smoothed1.ewm(span=3, adjust=False).mean()
    return stoch_macd_smoothed2

df['STC'] = schaff_trend_cycle(df['Close'])
df = df.dropna().reset_index(drop=True)

## Prepare Data for LSTM

In [ ]:
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
scaler = MinMaxScaler()
scaled_close = scaler.fit_transform(df[['Close']])
seq_len = 20
X, y = [], []
for i in range(len(scaled_close)-seq_len-1):
    X.append(scaled_close[i:i+seq_len])
    y.append(df['Target'].iloc[i+seq_len])
X, y = np.array(X), np.array(y)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Train LSTM

In [ ]:
model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
lstm_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
lstm_acc = accuracy_score(y_test, lstm_pred)
print('LSTM accuracy:', lstm_acc)

## STC Baseline

In [ ]:
stc_signal = (df['STC'] > 50).astype(int)
stc_pred = stc_signal.shift(1).dropna().iloc[seq_len:]
stc_true = df['Target'].iloc[seq_len+1:]
stc_acc = accuracy_score(stc_true, stc_pred)
print('STC baseline accuracy:', stc_acc)

## Comparison

In [ ]:
print('LSTM Accuracy:', lstm_acc)
print('STC Baseline Accuracy:', stc_acc)